In [1]:
from datetime import datetime
import requests
import sys
import numpy as np
import pandas as pd

sys.path.append('..')
import moonlander as ml

from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
btc = ml.core.Asset('ETH', 'USDT', '1-min', provider="Binance")

In [3]:
btc.fetch_historical_data(start="2021-05-27 00:00:00")

In [4]:
# btc.plot_candles(start="2016-01-01 00:00:00")

In [ ]:
btc.stream()

Connection established...
2021-05-29 02:24:59 | New candle received...
Plotting the recent complete candle
2021-05-29 02:25:59 | New candle received...
Plotting the recent complete candle
2021-05-29 02:26:59 | New candle received...
Plotting the recent complete candle
2021-05-29 02:27:59 | New candle received...
Plotting the recent complete candle
2021-05-29 02:28:59 | New candle received...


In [6]:
btc.plot_returns()

In [7]:
btc.plot_returns('hs')

In [9]:
btc.annualized_perf()

Return: nan% | Risk: nan%


## Strategy

In [8]:
class TradingStrategyBase:
    def __init__(self, data):
        self.data = data
        self.tracker = []
        
    def logic(self, account, lookback):
        date = lookback.date.values[-1]

        choice = np.random.randint(3)
        if choice == 0:
            account.enter_position('long', np.random.randint(0, account.current_capital), lookback.close.values[-1], date)
        
        elif choice == 1:
            account.close_position('short', np.random.randint(-1, account.current_capital), lookback.close.values[-1], date)
            
        else:
            print('Holding...')
    
    def backtest(self, amount = 1000):
        self.tracker = []
        buy_hold_units = int(amount/self.data.close.values[0])
        account = ml.account.LocalAccount(amount)

        # Enter backtest ---------------------------------------------  
        for index, today in self.data.iterrows():
            date = today['date']
            equity = account.total_value(today['close'], date)

            # Handle stop loss
            for p in account.positions:
                if p.type == "long":
                    if p.stop_hit(today['low']):
                        account.close_position(p, 1.0, today['low'])
                if p.type == "short":
                    if p.stop_hit(today['high']):
                        account.close_position(p, 1.0, today['high'])
            
            account.purge_positions()
            
            # Update account variables
            account.equity.append(equity)

            # Equity tracking
            self.tracker.append({'date': date, 
                                 'benchmark_equity' : today['close'],
                                 'strategy_equity' : equity})

            # Execute trading logic
            lookback = self.data[0:index+1]
            self.logic(account, lookback)
        # ------------------------------------------------------------

        

In [9]:
basicStrat = TradingStrategyBase(df)

In [10]:
basicStrat.backtest(2000)

Holding...
Holding...
Holding...
Holding...
Holding...
Holding...


ValueError: Error: Entry capital must be positive

In [105]:
basicStrat.tracker

[{'date': Timestamp('2016-01-01 00:00:00'),
  'benchmark_equity': 1999.9997102000002,
  'strategy_equity': 0.0},
 {'date': Timestamp('2016-01-01 02:00:00'),
  'benchmark_equity': 1991.1529181,
  'strategy_equity': 0.0},
 {'date': Timestamp('2016-01-01 04:00:00'),
  'benchmark_equity': 2010.2980009,
  'strategy_equity': 0.0},
 {'date': Timestamp('2016-01-01 06:00:00'),
  'benchmark_equity': 1997.1885800000002,
  'strategy_equity': 0.0},
 {'date': Timestamp('2016-01-01 08:00:00'),
  'benchmark_equity': 1988.9297367000001,
  'strategy_equity': 1625.2491918900002},
 {'date': Timestamp('2016-01-01 10:00:00'),
  'benchmark_equity': 1961.1767160000002,
  'strategy_equity': 108.18525960000001},
 {'date': Timestamp('2016-01-01 12:00:00'),
  'benchmark_equity': 2016.48065,
  'strategy_equity': 55.37546},
 {'date': Timestamp('2016-01-01 14:00:00'),
  'benchmark_equity': 1999.02592,
  'strategy_equity': 1.529728},
 {'date': Timestamp('2016-01-01 16:00:00'),
  'benchmark_equity': 1978.907047,
  'st

In [106]:
timestamps = [x['date'] for x in basicStrat.tracker]
buyHold = [x['benchmark_equity'] for x in basicStrat.tracker]
stratEq = [x['strategy_equity'] for x in basicStrat.tracker]

In [107]:
from bokeh.plotting import figure, ColumnDataSource, show
from bokeh.layouts import gridplot
from bokeh.models import CDSView, BooleanFilter, HoverTool
from bokeh.models.formatters import NumeralTickFormatter

p = figure(plot_width=800, plot_height=500, toolbar_location=None, x_axis_type="datetime", title="Strategy Performance")
p.grid.grid_line_alpha=0.3
p.line(x=timestamps, y=buyHold, line_color="navy")
p.line(x=timestamps, y=stratEq, line_color="red")
p.yaxis.axis_label = 'Close Price'
show(p)

In [108]:
datetime.now()

datetime.datetime(2021, 5, 26, 22, 37, 0, 299956)